In [1]:
import tensorflow as tf
import pandas as pd
import os

In [ ]:
# -------- CONFIG --------
IMG_SIZE = (1600, 3200)
BATCH_SIZE = 8  # You can tune this based on your RAM/GPU
EPOCHS = 10

# -------- PATHS -------- (update for local paths)
train_csv = "train.csv"
val_csv = "validate.csv"
test_csv = "test.csv"

test_dir = r"C:\Users\vance\Desktop\project_folder\p3\USD.v3i.coco\train"
test_dir = r"C:\Users\vance\Desktop\project_folder\p3\USD.v3i.coco\valid"
test_dir = r"C:\Users\vance\Desktop\project_folder\p3\USD.v3i.coco\test"

AUTOTUNE = tf.data.AUTOTUNE


# -------- READ DATAFRAME --------
def get_dataset_from_csv(csv_file, image_dir):
    df = pd.read_csv(csv_file)
    df['file_path'] = df['file_name'].apply(lambda x: os.path.join(image_dir, x))
    file_paths = df['file_path'].values
    labels = df['category_id'].values
    return file_paths, labels


# -------- PREPROCESS FUNCTION --------
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return img, label


# -------- BUILD TF.DATA PIPELINE --------
def build_dataset(file_paths, labels, is_training=True):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.map(process_path, num_parallel_calls=AUTOTUNE)
    if is_training:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds


# -------- LOAD DATASETS --------
train_files, train_labels = get_dataset_from_csv(train_csv, train_dir)
val_files, val_labels = get_dataset_from_csv(val_csv, val_dir)
test_files, test_labels = get_dataset_from_csv(test_csv, test_dir)

num_classes = len(set(train_labels))

train_ds = build_dataset(train_files, train_labels, is_training=True)
val_ds = build_dataset(val_files, val_labels, is_training=False)
test_ds = build_dataset(test_files, test_labels, is_training=False)


In [ ]:
# -------- BUILD MODEL -------- (change the layers!)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


# -------- TRAIN --------
model.fit(train_ds,
          validation_data=val_ds,
          epochs=EPOCHS)

# -------- EVALUATE --------
loss, acc = model.evaluate(test_ds)
print(f"\n✅ Test accuracy: {acc:.4f}")